In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
from sklearn.compose import make_column_transformer

In [2]:


headers = ['date', 'ignore', 'open', 'high', 'low', 'close', 'volume'  ]

week_start_date ='2021.01.17'
week_end_date = '2021.01.23'


In [3]:
# this cell checks if all the currency pairs have the same number of rows

total_rows = 0

for a in currency_pairs:
    current_currency_dataframe = pd.read_csv('recent_5min_data/' + a['name']+ str(5) + '.csv', names=headers  ) 
    
    # save current currency dataframe to memory to speedup the consecutive calls to the dataframe by saving to a variable
    #filter dataframe to only return needed rows for the week
    
    a['dataframe']= current_currency_dataframe.loc[  (current_currency_dataframe['date'] >= week_start_date)
                                                     & 
                                                      (current_currency_dataframe['date'] <= week_end_date)
                                                  ]
    
    #drops the first 24 columns so as to start analysis by 2am and not 12am
    #a['dataframe'] = a['dataframe'].iloc[24: , :]
    
    #reset the index of the dataframe to refresh it
    a['dataframe'].reset_index(drop=True, inplace=True)
    
    print('----------------------------',a['name'])
    
    
    current_currency_dataframe_row_ammount = len(a['dataframe'].index)
    
    if total_rows == 0:
        total_rows = current_currency_dataframe_row_ammount
       
        
    else:
        if total_rows != current_currency_dataframe_row_ammount:
            print('ERROR,   ERROR,    ERROR-------lenght of files in', a ,'are not the same with total rows: ', total_rows)
            total_rows = "lenght of rows is not the same"

print(total_rows)

---------------------------- USDCHF
---------------------------- GBPUSD
---------------------------- EURUSD
---------------------------- USDJPY
---------------------------- USDCAD
---------------------------- NZDUSD
---------------------------- GBPCHF
---------------------------- EURCHF
---------------------------- CHFJPY
---------------------------- CADCHF
---------------------------- NZDCHF
---------------------------- EURGBP
---------------------------- GBPJPY
---------------------------- GBPCAD
---------------------------- GBPAUD
---------------------------- EURJPY
---------------------------- EURCAD
---------------------------- EURAUD
---------------------------- EURNZD
---------------------------- AUDJPY
---------------------------- NZDJPY
---------------------------- AUDCAD
1439


In [4]:
timeframe = []
trade_values = []

for b in range(total_rows):
    
   
    # save current timefreame to timeframe list
    timeframe.append(currency_pairs[0]['dataframe'].loc[b]['date'] + ' '+ currency_pairs[0]['dataframe'].loc[b]['ignore']  )
    
    sum_of_all_open_trades = 0
    
    for c in currency_pairs:
        
        
        if c['direction'] == 'b':
            
            current_trade_value =  c['dataframe'].loc[b]['close'] - c['dataframe'].loc[0]['open']
            
        else:
            
            current_trade_value =  c['dataframe'].loc[0]['open'] -  c['dataframe'].loc[b]['close']
            
        
        pips_corrector = 100000

        if c['name'].endswith('JPY'):   # use this to correct multiplier of jpy pairs
            pips_corrector = 1000
        
        current_trade_value = current_trade_value * pips_corrector
        
        #print(c['name'], '---', current_trade_value )
        
        
        
        sum_of_all_open_trades = sum_of_all_open_trades + current_trade_value
        
       
        
    
    trade_values.append(sum_of_all_open_trades)
  
    


trade_values_dataframe = pd.DataFrame()

trade_values_dataframe['timeframe'] = timeframe
trade_values_dataframe['profit']= trade_values

trade_values_dataframe







,timeframe,profit
0,2021.01.18 00:00,-108.0
1,2021.01.18 00:05,-32.0
2,2021.01.18 00:10,-45.0
3,2021.01.18 00:15,15.0
4,2021.01.18 00:20,-50.0
...,...,...
1434,2021.01.22 23:30,-2302.0
1435,2021.01.22 23:35,-2199.0
1436,2021.01.22 23:40,-2305.0
1437,2021.01.22 23:45,-2230.0


In [5]:
trade_values_dataframe.to_csv('files/' + 'maximaldraw_down_and_up' + '.csv', index=False)